In [189]:
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
import pandas as pd
from numpy import int16

In [190]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [191]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

True

In [192]:
# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    dm.id_patient AS main_id,
    IF(TIMESTAMPDIFF(YEAR,
            dsd.nan_ki_dat_ou_fet,
            NOW()) >= 18,
        IF(dsd.a7_Ak_kiles_w_ap_viv_15_19 = '3'
                OR dsd.a1121_aktivite_pouw_rantre_kob_ou_vle_fe = '0'
                or dsd.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19 = '3'
                or dsd.eske_ou_bay_kob_pou_pran_swen_piti_ou_ayo = 'oui',
            'yes_sup18',
            'no18'),
        if(
			dsd.a7_ak_kils_w_ap_viv_10_14 = '6'
            OR dsd.a1121_aktivite_pouw_rantre_kob_ou_vle_fe = '0'
            OR dsd.c6b_kils_ki_peye_lekl_ou_10_14 = '5'
            or dsd.eske_ou_bay_kob_pou_pran_swen_piti_ou_ayo = 'oui',
            'yes_inf17'
            ,'no17'	)
		) AS muso_eligibility,
    dm.id_group AS actual_id_group,
    b.groups AS group_she_take_sessions,
    p.patient_code AS code,
    ben.last_name,
    ben.first_name,
    ben.dob,
    TIMESTAMPDIFF(YEAR, ben.dob, NOW()) AS age,
    b.pres AS number_of_different_topic,
    b.first_session_date,
    b.last_session_date,
    MAX(dhi.test_date) AS last_hiv_test_date,
    MAX(dhi.hiv_test_awareness_date) AS last_sensibilisation_hiv_test_date,
    GROUP_CONCAT(distinct dhi.hiv_test_acceptation, ',') acceptation_hiv_test,
    GROUP_CONCAT(DISTINCT dhi.test_result, ',') AS test_results,
    GROUP_CONCAT(DISTINCT ltlr.name, ',') AS test_results_with_label,
    MAX(dhi.condoms_reception_date) AS last_condoms_reception_date,
    group_concat(distinct dhi.has_been_sensibilize_for_condom, ',') as sensibilisation_condom,
    group_concat(distinct dhi.accept_condom, ',') as acceptation_condom,
    MAX(dhi.vbg_treatment_date) AS last_vbg_treatment_date,
    MAX(dhi.gynecological_care_date) AS last_gynecological_care_date,
    MAX(dhi.prep_awareness_date) AS last_sensibilisation_prep_date,
    MAX(dhi.prep_reference_date) AS last_reference_prep_date,
    MAX(dhi.prep_initiation_date) AS last_initiation_prep_date,
    GROUP_CONCAT(distinct dhi.prep_acceptation, ',') AS acceptation_prep,
    dg.name AS actual_group_name,
    dh.name AS actual_hub,
    lc.name AS actual_commune,
    dh.commune AS actual_commune_id,
    ld.name AS actual_departement,
    lc.departement AS actual_departement_id,
    IF(mgm.id_patient IS NOT NULL,
        'yes',
        'no') AS is_muso,
    IF(gb.case_id IS NOT NULL, 'yes', 'no') AS is_gardening,
    tf.*,
    dsd.*
FROM
    dream_member dm
        LEFT JOIN
    (SELECT 
        SUM(a.value = 'P') AS pres,
            a.id_patient,
            GROUP_CONCAT(DISTINCT a.id_group, ',') AS groups,
            MIN(a.date) AS first_session_date,
            MAX(a.date) AS last_session_date
    FROM
        (SELECT 
        dga.*, dgs.topic, dgs.date, dgs.id_group
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
    GROUP BY dga.id_patient , dgs.topic) a
    GROUP BY a.id_patient) b ON b.id_patient = dm.id_patient
        LEFT JOIN
    beneficiary ben ON ben.id_patient = dm.id_patient
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    caris_db.dream_hivinfos dhi ON dhi.id_patient = dm.id_patient
        LEFT JOIN
    lookup_testing_lab_result ltlr ON ltlr.id = dhi.test_result
        LEFT JOIN
    caris_db.dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
        LEFT JOIN
    dreams_surveys_data dsd ON dsd.case_id = dm.case_id
        LEFT JOIN
    tracking_familymember tf ON tf.id_patient = dm.id_patient
        LEFT JOIN
    muso_group_members mgm ON mgm.id_patient = dm.id_patient
        LEFT JOIN
    gardening_beneficiary gb ON gb.code_dreams = p.patient_code
GROUP BY dm.id_patient
'''

dreams_mastersheet = pd.read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query
dreams_mastersheet.to_excel('./dreams_services.xlsx',index=False,na_rep="NULL")

In [193]:
# close the pool of connection
engine.dispose()

## Work on the age aspect

In [194]:
# how many NA
dreams_mastersheet.age.isna().sum()

2

In [195]:
# turn to integer
dreams_mastersheet.age = dreams_mastersheet.age.fillna(-1000)
dreams_mastersheet.age = dreams_mastersheet.age.astype(int16)

In [196]:
dreams_mastersheet.age

0       -1000
1          16
2          19
3          20
4          19
         ... 
26801      11
26802      13
26803      11
26804      12
26805      11
Name: age, Length: 26806, dtype: int16

In [197]:
# tranche d'age classique pr les services agyw
# tranche age mineur majeur pour les services agyw
# ovc age
def tranche_age_classique(age):
    """age classique"""
    if age>=10 and age<=14:
        return "10-14"
    elif age>=15 and age<=19:
        return "15-19"
    elif age>=20 and age<=24:
        return "20-24"
    elif age>=25 and age<=29:
        return "25-29"
    else:
        return "not_valid_age"
        
def tranche_age_mineur_majeur(age):
    """minor and major age"""
    if age>=10 and age<=17:
        return "10-17"
    elif age>=18 and age<=19:
        return "18-19"
    elif age>=20 and age<=24:
        return "20-24"
    elif age>=25 and age<=29:
        return "25-29"
    else:
        return "not_valid_age"
    
def ovc_age(age):
    """ovc valid age"""
    if age>=10 and age<=14:
        return "10-14"
    if age>=15 and age<=17:
        return "15-17"
    elif age>=18 and age<=24:
        return "not_ovc_age"
    else:
        return "not_valid_age"

In [198]:
dreams_mastersheet['age_range'] = dreams_mastersheet.age.map(tranche_age_classique)
dreams_mastersheet['newage_range'] = dreams_mastersheet.age.map(tranche_age_mineur_majeur)
dreams_mastersheet['ovcage_range'] = dreams_mastersheet.age.map(ovc_age)

In [199]:
dreams_mastersheet.newage_range.unique()
dreams_mastersheet.age_range.unique()
dreams_mastersheet.ovcage_range.unique()

array(['not_valid_age', '10-17', '18-19', '20-24', '25-29'], dtype=object)

array(['not_valid_age', '15-19', '20-24', '10-14', '25-29'], dtype=object)

array(['not_valid_age', '15-17', 'not_ovc_age', '10-14'], dtype=object)

## Work on the interview date aspect

In [200]:
dreams_mastersheet.a1_dat_entvyou_a_ft_jjmmaa_egz_010817.isna().sum()

0

In [201]:
dreams_mastersheet["date_entevyou"] = pd.to_datetime( dreams_mastersheet.a1_dat_entvyou_a_ft_jjmmaa_egz_010817)

In [202]:
dreams_mastersheet.date_entevyou

0       2020-07-09
1       2019-12-18
2       2019-12-18
3       2019-12-18
4       2019-12-18
           ...    
26801   2020-10-10
26802   2021-04-09
26803   2021-03-16
26804   2021-04-09
26805   2021-02-12
Name: date_entevyou, Length: 26806, dtype: datetime64[ns]

In [203]:
def calculation_fiscalYear21(date):
    if date.year == 2021 and date.month>=1 and date.month<=3:
        return "FY21Q2"
    elif date.year == 2020 and date.month>=10 and date.month<=12:
        return "FY21Q1"
    elif date.year == 2021 and date.month>=4 and date.month<=6:
        return "FY21Q3"
    elif date.year == 2021 and date.month>=7 and date.month<=9:
        return "FY21Q4"
    elif date.year == 2020 and date.month>=4 and date.month<=6:
        return "FY20Q3"
    elif date.year == 2020 and date.month>=7 and date.month<=9:
        return "FY20Q4"
    else:
        return "not_valid"

def fiscalYear21(mod):
    if mod == "FY20Q3" or mod == "FY20Q4":
        return "Q3fy20-Q4fy20"
    else:
        return mod

    
def validTimeOnSystem(date):
    if date>= datetime.strptime("2020-04-01","%Y-%m-%d") and date<= datetime.now():
        return "required_Time_on"
    else:
        return "not_valid_time_on"
        
    

    
def between_now_date_entevyou(date):
    return (datetime.now().year - date.year) * 12 + (datetime.now().month - date.month)




def agywPeriods(months):
    if months <= 6:
        return "0-6 months"
    elif months>=7 and months<=12:
        return "7-12 months"
    elif months>=13 and months<=24:
        return "13-24 months"
    else:
        return "25+ months"

In [204]:
dreams_mastersheet["id_fiscal_year"] = dreams_mastersheet.date_entevyou.map(calculation_fiscalYear21)
dreams_mastersheet["fiscal_year"] = dreams_mastersheet.id_fiscal_year.map(fiscalYear21)
dreams_mastersheet["timeOn_system"] = dreams_mastersheet.date_entevyou.map(validTimeOnSystem)
dreams_mastersheet["months_now_dateEntevyou"] = dreams_mastersheet.date_entevyou.map(between_now_date_entevyou)
dreams_mastersheet["agyw_period_range"] = dreams_mastersheet.months_now_dateEntevyou.map(agywPeriods)

In [205]:
dreams_mastersheet.id_fiscal_year.unique()
dreams_mastersheet.fiscal_year.unique()
dreams_mastersheet.timeOn_system.unique()
#dreams_mastersheet.months_now_dateEntevyou.unique()
dreams_mastersheet.agyw_period_range.unique()

array(['FY20Q4', 'not_valid', 'FY20Q3', 'FY21Q1', 'FY21Q2', 'FY21Q3',
       'FY21Q4'], dtype=object)

array(['Q3fy20-Q4fy20', 'not_valid', 'FY21Q1', 'FY21Q2', 'FY21Q3',
       'FY21Q4'], dtype=object)

array(['required_Time_on', 'not_valid_time_on'], dtype=object)

array(['13-24 months', '25+ months', '7-12 months', '0-6 months'],
      dtype=object)

## Curriculum services aspect

In [206]:
dreams_mastersheet.number_of_different_topic.isna().sum()

1765

In [207]:
dreams_mastersheet.number_of_different_topic = dreams_mastersheet.number_of_different_topic.fillna(-1000)
dreams_mastersheet.number_of_different_topic = dreams_mastersheet.number_of_different_topic.astype(int16)

In [208]:
dreams_mastersheet.number_of_different_topic.unique()

array([-1000,    18,    10,    13,    17,    14,    15,     6,    16,
          11,     3,     2,     7,     9,     8,     1,     4,    12,
           5], dtype=int16)

In [209]:
def curriculum_atLeastOneService(topics):
    return "servis_auMoins_1fois" if topics>=1 and topics<=19 else "zero_services_curriculum" #topics<=18


def status_curriculum(topics):
    if topics>=1 and topics<=13:
        return "curriculum incomplet"
    elif topics>=14 and topics<=19: #topics<=18
        return "curriculum complet"
    else:
        return "non-recu"


### - Au moins un services du Curriculum

In [210]:
dreams_mastersheet['curriculum_servis_auMoins_1fois'] = dreams_mastersheet.number_of_different_topic.map(curriculum_atLeastOneService)

In [211]:
dreams_mastersheet.curriculum_servis_auMoins_1fois.unique()

array(['zero_services_curriculum', 'servis_auMoins_1fois'], dtype=object)

### - Curriculum Status

In [212]:
dreams_mastersheet['curriculum'] = dreams_mastersheet.number_of_different_topic.map(status_curriculum)

In [213]:
dreams_mastersheet.curriculum.unique()
# complet est service primaire pour 10-14

array(['non-recu', 'curriculum complet', 'curriculum incomplet'],
      dtype=object)

### Curriculum date trimestre

In [214]:
def id_quarter_services(date):
    if type(date) == type(pd.NaT):
        return 'errata'
    if (type(date) != type(pd.NaT)) and (date.year == 2021 and date.month>=1 and date.month<=3):
        return "FY21Q2"
    elif (type(date) != type(pd.NaT)) and (date.year == 2020 and date.month>=10 and date.month<=12):
        return "FY21Q1"
    elif (type(date) != type(pd.NaT)) and (date.year == 2021 and date.month>=4 and date.month<=6):
        return "FY21Q3"
    elif (type(date) != type(pd.NaT)) and (date.year == 2021 and date.month>=7 and date.month<=9):
        return "FY21Q4"
    elif (type(date) != type(pd.NaT)) and (date.year == 2020 and date.month>=1 and date.month<=3):
        return "FY20Q2"
    elif (type(date) != type(pd.NaT)) and (date.year == 2019 and date.month>=10 and date.month<=12):
        return "FY20Q1"
    elif (type(date) != type(pd.NaT)) and (date.year == 2020 and date.month>=4 and date.month<=6):
        return "FY20Q3"
    elif (type(date) != type(pd.NaT)) and (date.year == 2020 and date.month>=7 and date.month<=9):
        return "FY20Q4"
    elif (type(date) != type(pd.NaT)) and (date.year == 2019 and date.month>=1 and date.month<=3):
        return "FY19Q2"
    elif (type(date) != type(pd.NaT)) and (date.year == 2018 and date.month>=10 and date.month<=12):
        return "FY19Q1"
    elif (type(date) != type(pd.NaT)) and (date.year == 2019 and date.month>=4 and date.month<=6):
        return "FY19Q3"
    elif (type(date) != type(pd.NaT)) and (date.year == 2019 and date.month>=7 and date.month<=9):
        return "FY19Q4"
    else:
        return "not_valid_fy"

In [215]:
dreams_mastersheet.first_session_date.isna().sum()

1765

In [216]:
dreams_mastersheet.last_session_date.isna().sum()

1765

In [217]:
dreams_mastersheet.first_session_date = dreams_mastersheet.first_session_date.fillna('0000-00-00')
dreams_mastersheet.last_session_date = dreams_mastersheet.last_session_date.fillna('0000-00-00')

dreams_mastersheet["curriculum_date_debut"] = pd.to_datetime( dreams_mastersheet.first_session_date,errors='coerce')
dreams_mastersheet["curriculum_date_end"] = pd.to_datetime( dreams_mastersheet.last_session_date,errors='coerce')

In [218]:
dreams_mastersheet.curriculum_date_debut.dt.year.unique()

array([  nan, 2020., 2021., 2019.])

In [219]:
dreams_mastersheet.curriculum_date_end.dt.year.unique()

array([  nan, 2020., 2021.])

In [220]:
dreams_mastersheet['curriculum_date_debut_fy'] = dreams_mastersheet.curriculum_date_debut.map(id_quarter_services)
dreams_mastersheet['curriculum_date_end_fy'] = dreams_mastersheet.curriculum_date_end.map(id_quarter_services)

In [221]:
dreams_mastersheet.curriculum_date_debut_fy.unique()

array(['errata', 'FY20Q4', 'FY20Q3', 'FY20Q2', 'FY21Q3', 'FY21Q2',
       'FY19Q3', 'FY21Q1', 'FY21Q4'], dtype=object)

In [222]:
dreams_mastersheet.curriculum_date_end_fy.unique()

array(['errata', 'FY20Q4', 'FY20Q3', 'FY20Q2', 'FY21Q4', 'FY21Q2',
       'FY21Q3', 'FY21Q1'], dtype=object)

# hts, vbg, condoms, gyneco, muso, garden,prep

In [294]:
def valid_services(date):
    if type(date) == type(pd.NaT):
        return 'errata'
    elif (type(date) != type(pd.NaT))and(date.year==2020 or date.year==2021):
        return 'tested_on_given_date'
    else:
        return 'not_valid_date'
    
def acceptation_services(choices):
    if choices == "0,":
        return "refus"
    elif choices == '0,,1,' or choices == '1,':
        return "accept"
    else:
        return "not_valid"

def sensibilisation_services(choices):
    if choices == "0,":
        return "non-sens"
    elif choices == '0,,1,' or choices == '1,':
        return "sens"
    else:
        return "not_valid"

###  PREP

In [297]:
dreams_mastersheet.last_initiation_prep_date.isna().sum()
dreams_mastersheet.acceptation_prep.isna().sum()
dreams_mastersheet.last_sensibilisation_prep_date.isna().sum()
dreams_mastersheet.last_reference_prep_date.isna().sum()

23632

0

0

0

In [295]:
dreams_mastersheet.acceptation_prep = dreams_mastersheet.acceptation_prep.fillna("didnt")
dreams_mastersheet["acc_prep"] = dreams_mastersheet.acceptation_prep.map(acceptation_services)

In [300]:
dreams_mastersheet.last_sensibilisation_prep_date = dreams_mastersheet.last_sensibilisation_prep_date.fillna('0000-00-00')
dreams_mastersheet["sens_prep_date"] = pd.to_datetime( dreams_mastersheet.last_sensibilisation_prep_date,errors='coerce')

dreams_mastersheet.last_reference_prep_date = dreams_mastersheet.last_reference_prep_date.fillna('0000-00-00')
dreams_mastersheet["ref_prep_date"] = pd.to_datetime( dreams_mastersheet.last_reference_prep_date,errors='coerce')

dreams_mastersheet.last_initiation_prep_date = dreams_mastersheet.last_initiation_prep_date.fillna('0000-00-00')
dreams_mastersheet["init_prep_date"] = pd.to_datetime( dreams_mastersheet.last_initiation_prep_date,errors='coerce')

In [302]:
dreams_mastersheet.init_prep_date.dt.year.unique()
dreams_mastersheet.acc_prep.unique()
dreams_mastersheet.ref_prep_date.dt.year.unique()
dreams_mastersheet.sens_prep_date.dt.year.unique()

array([  nan, 2021.])

array(['not_valid', 'refus', 'accept'], dtype=object)

array([  nan, 2021.])

array([  nan, 2021.])

## hts vbg gyneco condoms

In [303]:
print("hts")
dreams_mastersheet.last_hiv_test_date.isna().sum()
dreams_mastersheet.last_sensibilisation_hiv_test_date.isna().sum()
dreams_mastersheet.acceptation_hiv_test.isna().sum()

print("condom")
dreams_mastersheet.last_condoms_reception_date.isna().sum()
dreams_mastersheet.sensibilisation_condom.isna().sum()
dreams_mastersheet.acceptation_condom.isna().sum()

print("vbg")
dreams_mastersheet.last_vbg_treatment_date.isna().sum()

print("gyneco")
dreams_mastersheet.last_gynecological_care_date.isna().sum()


hts


0

24801

24797

condom


...

23607

vbg


0

gyneco


0

In [311]:
dreams_mastersheet.acceptation_condom = dreams_mastersheet.acceptation_condom.fillna("didnt")
dreams_mastersheet["acc_condom"] = dreams_mastersheet.acceptation_condom.map(acceptation_services)

dreams_mastersheet.acceptation_hiv_test = dreams_mastersheet.acceptation_hiv_test.fillna("didnt")
dreams_mastersheet["acc_hiv_test"] = dreams_mastersheet.acceptation_hiv_test.map(acceptation_services)

In [312]:
dreams_mastersheet.sensibilisation_condom = dreams_mastersheet.sensibilisation_condom.fillna("didnt")
dreams_mastersheet["sens_condom"] = dreams_mastersheet.sensibilisation_condom.map(sensibilisation_services)


In [306]:
dreams_mastersheet.last_hiv_test_date = dreams_mastersheet.last_hiv_test_date.fillna('0000-00-00')
dreams_mastersheet["hts_date"] = pd.to_datetime( dreams_mastersheet.last_hiv_test_date,errors='coerce')

dreams_mastersheet.last_sensibilisation_hiv_test_date = dreams_mastersheet.last_sensibilisation_hiv_test_date.fillna('0000-00-00')
dreams_mastersheet["sens_hts_date"] = pd.to_datetime( dreams_mastersheet.last_sensibilisation_hiv_test_date,errors='coerce')


dreams_mastersheet.last_condoms_reception_date = dreams_mastersheet.last_condoms_reception_date.fillna('0000-00-00')
dreams_mastersheet['condoms_date'] = pd.to_datetime(dreams_mastersheet.last_condoms_reception_date,errors='coerce')

dreams_mastersheet.last_vbg_treatment_date = dreams_mastersheet.last_vbg_treatment_date.fillna('0000-00-00')
dreams_mastersheet['vbg_date'] = pd.to_datetime(dreams_mastersheet.last_vbg_treatment_date,errors='coerce')

dreams_mastersheet.last_gynecological_care_date = dreams_mastersheet.last_gynecological_care_date.fillna('0000-00-00')
dreams_mastersheet['gyneco_date'] = pd.to_datetime(dreams_mastersheet.last_gynecological_care_date,errors='coerce')


In [314]:
dreams_mastersheet.sens_hts_date.dt.year.unique()
dreams_mastersheet.hts_date.dt.year.unique()
dreams_mastersheet.condoms_date.dt.year.unique()
dreams_mastersheet.vbg_date.dt.year.unique()
dreams_mastersheet.gyneco_date.dt.year.unique()

dreams_mastersheet.acc_condom.unique()
dreams_mastersheet.acc_hiv_test.unique()
dreams_mastersheet.sens_condom.unique()

array([  nan, 2021.])

array([  nan, 2020., 2021.])

array([  nan, 2020., 2021.])

array([  nan, 2020., 2021.])

array([  nan, 2020., 2021.])

array(['not_valid', 'refus', 'accept'], dtype=object)

array(['not_valid', 'refus', 'accept'], dtype=object)

array(['not_valid', 'non-sens', 'sens'], dtype=object)

## validation date type services 

In [233]:
dreams_mastersheet['hts'] = dreams_mastersheet.hts_date.map(hcvg_valid_services)
dreams_mastersheet['condoms'] = dreams_mastersheet.condoms_date.map(hcvg_valid_services)
dreams_mastersheet['vbg'] = dreams_mastersheet.vbg_date.map(hcvg_valid_services)
dreams_mastersheet['gyneco'] = dreams_mastersheet.gyneco_date.map(hcvg_valid_services)

dreams_mastersheet['hts_fy'] = dreams_mastersheet.hts_date.map(id_quarter_services)
dreams_mastersheet['condoms_fy'] = dreams_mastersheet.condoms_date.map(id_quarter_services)
dreams_mastersheet['vbg_fy'] = dreams_mastersheet.vbg_date.map(id_quarter_services)
dreams_mastersheet['gyneco_fy'] = dreams_mastersheet.gyneco_date.map(id_quarter_services)

In [234]:
dreams_mastersheet.hts.unique()

array(['errata', 'tested_on_given_date'], dtype=object)

In [235]:
dreams_mastersheet.condoms.unique()

array(['errata', 'tested_on_given_date'], dtype=object)

In [236]:
dreams_mastersheet.vbg.unique()

array(['errata', 'tested_on_given_date'], dtype=object)

In [237]:
dreams_mastersheet.gyneco.unique()

array(['errata', 'tested_on_given_date'], dtype=object)

In [238]:
dreams_mastersheet.condoms_fy.unique()

array(['errata', 'FY20Q4', 'FY20Q2', 'FY21Q4', 'FY21Q3', 'FY20Q3',
       'FY21Q1', 'FY21Q2'], dtype=object)

In [239]:
dreams_mastersheet.hts_fy.unique()

array(['errata', 'FY20Q2', 'FY20Q4', 'FY21Q4', 'FY20Q3', 'FY21Q3',
       'FY21Q1', 'FY21Q2'], dtype=object)

In [240]:
dreams_mastersheet.vbg_fy.unique()

array(['errata', 'FY20Q4', 'FY20Q3', 'FY21Q4', 'FY21Q3'], dtype=object)

In [241]:
dreams_mastersheet.gyneco_fy.unique()

array(['errata', 'FY20Q4', 'FY21Q4'], dtype=object)

## Post Care aspect

In [242]:
def post_care_app(df):
    return 'service_gyneco_vbg' if (df.vbg=="tested_on_given_date") or (df.gyneco=='tested_on_given_date') else 'no'

In [243]:
dreams_mastersheet['post_care_treatment'] = dreams_mastersheet.apply(lambda df: post_care_app(df),axis=1)

In [244]:
dreams_mastersheet.post_care_treatment.unique()

array(['no', 'service_gyneco_vbg'], dtype=object)

## Muso Gardening =  socio eco approach Aspect

In [245]:
dreams_mastersheet.is_muso.isna().sum()

0

In [246]:
dreams_mastersheet.is_gardening.isna().sum()

0

In [247]:
dreams_mastersheet.is_muso.unique()

array(['no', 'yes'], dtype=object)

In [248]:
dreams_mastersheet.is_gardening.unique()

array(['no', 'yes'], dtype=object)

In [249]:
def socioEco_app(df):
    return 'service_muso_gardening' if (df.is_muso=='yes') or (df.is_gardening=='yes') else 'no'

In [250]:
dreams_mastersheet['socio_eco_app'] = dreams_mastersheet.apply(lambda df: socioEco_app(df),axis=1)

In [251]:
dreams_mastersheet.socio_eco_app.unique()

array(['no', 'service_muso_gardening'], dtype=object)

## Au moins un service recu Aspect

In [252]:
def unServiceDreams(df):
     return '1_services_dreams_recus' if (df.curriculum_servis_auMoins_1fois == "servis_auMoins_1fois") or (df.condoms=='tested_on_given_date') or (df.hts=='tested_on_given_date') or (df.post_care_treatment=="service_gyneco_vbg") or (df.socio_eco_app=="service_muso_gardening") else 'no'

In [253]:
dreams_mastersheet['recevoir_1services'] = dreams_mastersheet.apply(lambda df: unServiceDreams(df),axis=1)

In [254]:
dreams_mastersheet.recevoir_1services.unique()

array(['no', '1_services_dreams_recus'], dtype=object)

## Les Services Primaires

In [255]:
def service_primaire_10_14(df):
    return 'curriculum-servis' if (df.curriculum=="curriculum complet" and df.age_range == "10-14") else "no"

def service_primaire_15_19(df):
    return 'condoms&curriculum' if (df.curriculum=="curriculum complet" and df.age_range == "15-19" and df.condoms=='tested_on_given_date') else 'no'

def service_primaire_20_24(df):
    return 'condoms&hts&curriculum' if (df.curriculum=="curriculum complet" and df.condoms=='tested_on_given_date' and df.hts=='tested_on_given_date' and df.age_range == "20-24") else 'no'

def new_service_primaire_20_24(df):
    return 'condoms&curriculum' if (df.curriculum=="curriculum complet" and df.condoms=='tested_on_given_date' and df.age_range == "20-24") else 'no'

In [256]:
dreams_mastersheet['ps_10_14'] = dreams_mastersheet.apply(lambda df: service_primaire_10_14(df),axis=1)
dreams_mastersheet['ps_15_19'] = dreams_mastersheet.apply(lambda df: service_primaire_15_19(df), axis=1)
dreams_mastersheet['ps_20_24'] = dreams_mastersheet.apply(lambda df: new_service_primaire_20_24(df), axis=1)

In [257]:
dreams_mastersheet.ps_10_14.unique()

array(['no', 'curriculum-servis'], dtype=object)

In [258]:
dreams_mastersheet.ps_15_19.unique()

array(['no', 'condoms&curriculum'], dtype=object)

In [259]:
dreams_mastersheet.ps_20_24.unique()

array(['no', 'condoms&curriculum'], dtype=object)

## Screening

In [260]:
dreams_mastersheet.total.unique()

array([30, 27, 46, 34, 24, 45, 50, 20, 39, 42, 22, 10, 19, 47, 52, 15, 21,
       35, 54, 25, 29, 36, 56, 40, 53, 37, 32, 26, 14, 18, 49, 38, 43, 28,
       16, 17, 31, 23, 65, 41, 62, 33,  9, 57, 55,  3, 44, 48, 68, 51,  5,
       58, 63, 72, 66, 67, 74, 61, 60, 70, 64, 59, 12, 13, 75, 71, 69, 81,
       86, 73, 77, 83, 78, 80, 84, 76], dtype=int64)

In [261]:
def isAGYW(total):
    return 'eligible' if total>=14 else 'no_eligible'

In [262]:
dreams_mastersheet['score_eligible_AGYW'] = dreams_mastersheet.total.map(isAGYW)

In [263]:
dreams_mastersheet.score_eligible_AGYW.unique()

array(['eligible', 'no_eligible'], dtype=object)

In [264]:
def curriculum_condense(curriculum):
    return "curriculum_completed" if curriculum == "curriculum complet" else "curriculum_inc"

In [265]:
dreams_mastersheet["dreams_curriculum"] = dreams_mastersheet.curriculum.map(curriculum_condense)

## GET the complete EXCEL workbook

In [266]:
dreams_mastersheet.to_excel('./LayeringOF_services.xlsx',index=False,na_rep="NULL")

---